In [2]:
from google.colab import drive
drive.mount('/content/drive')
current_path='/content/drive/MyDrive'

Mounted at /content/drive


In [3]:
rootdir='/content/drive/MyDrive/23-similarity/data'
targetdir='/content/drive/MyDrive/23-similarity/data_preprocess'

In [ ]:
!rm -rf /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0001


In [8]:
import cv2
import os

def video_to_images(video_path,img_paph):
  # Read the video from specified path
  cam = cv2.VideoCapture(video_path)
  currentframe = 0
  count=0
  no_skip_count=0
  while(True):

      # reading from frame
      ret,frame = cam.read()

      if ret:
          no_skip_count+=1
          if no_skip_count%2!=0:
              continue

          # if video is still left continue creating images
          count+=1
          name = 'img_{:05d}.jpg'.format(count)
          print ('Creating...' + name)

          # writing the extracted images
          cv2.imwrite(os.path.join(img_paph,name), frame)

          # increasing counter so that it will
          # show how many frames are created
          currentframe += 1
      else:
          break

  # Release all space and windows once done
  cam.release()
  cv2.destroyAllWindows()

In [15]:
import os
import shutil
## Iterate over all folders
for folder in os.listdir(rootdir):
    if folder.endswith('.txt'):
      continue
    if folder.endswith('.rar'):
      continue
    subfolder = os.path.join(rootdir, folder)
    targetsubfolder=os.path.join(targetdir,folder)
    if 'Bor' in folder:
      continue
    if 'Mystical_robot' in folder:
      continue
    if 'Disco_robot' in folder:
      continue
    if 'Happy' in folder:
      continue
    count=0
    os.makedirs(targetsubfolder,exist_ok=True)
    ## Iterate over all videos in each folder
    print(folder,"******************")
    for video in os.listdir(subfolder):
        count+=1
        ## Create a new subfolder with the same name as the video
        video_path = os.path.join(subfolder, video)
        targetsubsubfolder=os.path.join(targetsubfolder,'{:04d}'.format(count))
        os.makedirs(targetsubsubfolder,exist_ok=True)
        ## Extract frames from the video
        video_to_images(video_path,targetsubsubfolder)


Streaming output truncated to the last 5000 lines.
Creating...img_00071.jpg
Creating...img_00072.jpg
Creating...img_00073.jpg
Creating...img_00074.jpg
Creating...img_00075.jpg
Creating...img_00076.jpg
Creating...img_00077.jpg
Creating...img_00078.jpg
Creating...img_00079.jpg
Creating...img_00080.jpg
Creating...img_00081.jpg
Creating...img_00082.jpg
Creating...img_00083.jpg
Creating...img_00084.jpg
Creating...img_00085.jpg
Creating...img_00086.jpg
Creating...img_00087.jpg
Creating...img_00088.jpg
Creating...img_00089.jpg
Creating...img_00090.jpg
Creating...img_00091.jpg
Creating...img_00092.jpg
Creating...img_00093.jpg
Creating...img_00094.jpg
Creating...img_00095.jpg
Creating...img_00096.jpg
Creating...img_00097.jpg
Creating...img_00098.jpg
Creating...img_00099.jpg
Creating...img_00100.jpg
Creating...img_00101.jpg
Creating...img_00102.jpg
Creating...img_00103.jpg
Creating...img_00104.jpg
Creating...img_00105.jpg
Creating...img_00106.jpg
Creating...img_00107.jpg
Creating...img_00108.jpg

In [4]:
import os
with open(os.path.join(rootdir,'label.txt'),'w') as f:
  count=-1
  for folder in os.listdir(targetdir):
    count+=1
    subfolder = os.path.join(targetdir, folder)
    for video in os.listdir(subfolder):
        print(video)
        targetsubsubfolder=os.path.join(subfolder,video)
        img_num = len(os.listdir(targetsubsubfolder))
        class_num = count
        f.write(os.path.join(folder,video)+' '+str(img_num)+' '+str(count)+'\n')


0002
0003
0004
0005
0006
0001
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0021
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0021
0022
0018
0004
0019
0005
0020
0007
0001
0002
0003
0006
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018


In [11]:
import sys
sys.path.append("/content/drive/MyDrive/23-similarity/similarity-assessment/i3d")

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
from i3d import I3D
import numpy as np
from matplotlib import pyplot as plt

In [14]:
from video_dataset import  VideoFrameDataset, ImglistToTensor
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os


In [15]:
videos_root="/content/drive/MyDrive/23-similarity/data_preprocess"
annotation_file="/content/drive/MyDrive/23-similarity/data_preprocess/label.txt"


In [16]:
"""
Ignore this function and look at "main" below.
"""
def plot_video(rows, cols, frame_list, plot_width, plot_height, title: str):
    fig = plt.figure(figsize=(plot_width, plot_height))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(rows, cols),  # creates 2x2 grid of axes
                     axes_pad=0.3,  # pad between axes in inch.
                     )

    for index, (ax, im) in enumerate(zip(grid, frame_list)):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_title(index)
    plt.suptitle(title)
    plt.show()


In [28]:
dataset = VideoFrameDataset(
        root_path=videos_root,
        annotationfile_path=annotation_file,
        num_segments=5,
        frames_per_segment=1,
        imagefile_template='img_{:05d}.jpg',
        transform=None,
        test_mode=False
)
print(len(dataset))
'''
sample = dataset[0]
frames = sample[0]  # list of PIL images
label = sample[1]   # integer label

plot_video(rows=1, cols=5, frame_list=frames, plot_width=15., plot_height=3.,
            title='Evenly Sampled Frames, No Video Transform')
'''


""" DEMO 3 CONTINUED: DATALOADER """
dataloader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
'''
for epoch in range(10):
    for video_batch, labels in dataloader:
        """
        Insert Training Code Here
        """
        print(labels)
        print("\nVideo Batch Tensor Size:", video_batch.size())
        print("Batch Labels Size:", labels.size())
        break
    break
'''


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0002 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0003 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0004 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0005 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0006 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0001 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/MyDrive/23-similarity/data_preprocess/Mystical_robot/0007 seems to have zero RGB frames on disk!


Dataset Warning: video /content/drive/My

'\nfor epoch in range(10):\n    for video_batch, labels in dataloader:\n        """\n        Insert Training Code Here\n        """\n        print(labels)\n        print("\nVideo Batch Tensor Size:", video_batch.size())\n        print("Batch Labels Size:", labels.size())\n        break\n    break\n'

In [25]:
print(len(dataloader))


74


In [23]:
for epoch in range(10):
    for video_batch, labels in dataloader:
      print(labels)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


ValueError: ignored